In [2]:
from edgeml import edgeml
import pandas as pd
import numpy as np
import datetime
import seaborn as sns

In [3]:
project=edgeml.getProject("https://app.edge-ml.org","dSCctVkGbmT/tMoPX4u6/2lzjQ82zWLweyUpBP7C/oYeXtNGi9iyiwMXr+JTbBzh+Yk1Oh/kJZiiGM/y6HLj2Q==")

fetching project...


In [4]:
#timestamp:{x0,y0,label}
data={}

for dataset in project["datasets"]:
  for labels in dataset["labels"]:
    for label in labels: #TODO I do not understand the API here
      start=datetime.datetime.utcfromtimestamp(label["start"]/1000)
      if start not in data:
        data[start]={("meta",None,k):v for k,v in dataset["metaData"].items()}

      data[start][("label",label["labelingName"],label["name"])]=1
      end=datetime.datetime.utcfromtimestamp(label["end"]/1000)
      if end not in data:
        data[end]={("meta",None,k):v for k,v in dataset["metaData"].items()}

      data[end][("label",label["labelingName"],label["name"])]=0
  for sensor in dataset["sensors"]:
    for dataPoint in sensor['data']:
      timestamp=datetime.datetime.utcfromtimestamp(dataPoint["timestamp"]/1000)
      if timestamp not in data:
        data[timestamp]={("meta",None,k):v for k,v in dataset["metaData"].items()}

      data[timestamp][("data",None,sensor["name"])]=dataPoint["datapoint"]

# subject id und andere labels fehlen noch.

In [5]:
df=pd.DataFrame.from_dict(data,"index").sort_index()
df.columns = pd.MultiIndex.from_tuples(df.columns)
cols=df.columns.values
cols.sort()
df=df[cols]
df

data                                              \
                           NaN                                               
                         alpha   beta  gamma     x    x0     y    y0     z   
2022-08-11 10:37:02.815  -3.19   0.38  -1.16 -0.23  0.00 -6.99  0.01 -6.89   
2022-08-11 10:37:02.831  -4.89  -1.07  -0.50 -0.22  0.00 -6.92  0.07 -6.87   
2022-08-11 10:37:02.847  -5.93  -1.34  -0.31 -0.23  0.00 -6.92  0.05 -7.01   
2022-08-11 10:37:02.864  -5.51   0.06  -0.37 -0.24 -0.01 -6.96  0.01 -7.08   
2022-08-11 10:37:02.881  -4.69   0.42  -0.97 -0.30 -0.07 -6.98 -0.02 -7.11   
...                        ...    ...    ...   ...   ...   ...   ...   ...   
2022-08-11 10:40:45.743 -10.91  30.91   3.49  1.44  0.24 -7.57  0.52 -5.10   
2022-08-11 10:40:45.760   1.91   2.69  10.32  1.38  0.17 -8.07  0.01 -5.45   
2022-08-11 10:40:45.841   5.05  -9.04   2.71  0.77 -0.14 -8.08  0.15 -5.14   
2022-08-11 10:40:45.842   5.05  -9.04   2.71  0.77 -0.14 -8.08  0.15 -5.14   
2022-08-11 10:40:45.860   9.20  -5.59  -1.93  0.63 -0.27 -7.93  0.31 -5.06   

                                   label                     meta          \
                              activities                      NaN           
                           z0    calling chatting reading browser  mobile   
2022-08-11 10:37:02.815 -0.02        NaN      NaN     1.0  Safari  iPhone   
2022-08-11 10:37:02.831  0.01        NaN      NaN     NaN  Safari  iPhone   
2022-08-11 10:37:02.847 -0.12        NaN      NaN     NaN  Safari  iPhone   
2022-08-11 10:37:02.864 -0.18        NaN      NaN     NaN  Safari  iPhone   
2022-08-11 10:37:02.881 -0.20        NaN      NaN     NaN  Safari  iPhone   
...                       ...        ...      ...     ...     ...     ...   
2022-08-11 10:40:45.743  0.32        NaN      NaN     NaN  Safari  iPhone   
2022-08-11 10:40:45.760 -0.03        NaN      NaN     NaN  Safari  iPhone   
2022-08-11 10:40:45.841  0.11        NaN      NaN     NaN  Safari  iPhone   
2022-08-11 10:40:45.842  0.11        NaN      NaN     NaN  Safari  iPhone   
2022-08-11 10:40:45.860  0.17        NaN      NaN     NaN  Safari  iPhone   

                                       
                                       
                        participantId  
2022-08-11 10:37:02.815         1086e  
2022-08-11 10:37:02.831         1086e  
2022-08-11 10:37:02.847         1086e  
2022-08-11 10:37:02.864         1086e  
2022-08-11 10:37:02.881         1086e  
...                               ...  
2022-08-11 10:40:45.743         1086e  
2022-08-11 10:40:45.760         1086e  
2022-08-11 10:40:45.841         1086e  
2022-08-11 10:40:45.842         1086e  
2022-08-11 10:40:45.860         1086e  

[10992 rows x 15 columns]